In [ ]:
import pandas as pd
import os
from pyparsing import C
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTForImageClassification
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 参数配置
CFG = {
	'data_path': 'chinese-medicine-image',
	'csv_name': 'train_labels.csv',
	'batch_size': 16,        # 根据显存调整
	'num_workers': 0,
	'num_epochs': 30,
	'lr': 3e-4,
	'image_size': 224,
	'num_classes': 881,
	'device': torch.device('cuda'),
	'seed': 42
}

# 数据预处理
train_transform = transforms.Compose([
	transforms.RandomResizedCrop(CFG['image_size']),
	transforms.RandomHorizontalFlip(),
	transforms.ColorJitter(0.2, 0.2, 0.2),
	transforms.ToTensor(),
	transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
	transforms.Resize(256),
	transforms.CenterCrop(CFG['image_size']),
	transforms.ToTensor(),
	transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 自定义数据集
class MedicineDataset(Dataset):
	def __init__(self, df, transform=None):
		self.df = df
		self.transform = transform
		
	def __len__(self):
		return len(self.df)
	
	def __getitem__(self, idx):
		img_path = self.df.iloc[idx, 0]
		label = self.df.iloc[idx, 1] - 1  # 标签转换为0-based
		
		# 关键断言
		assert 0 <= label < CFG['num_classes'], \
			f"无效标签：{label+1} (应为1-{CFG['num_classes']}), 位置：{idx}"
		img = Image.open(img_path).convert('RGB')
		if self.transform:
			img = self.transform(img)
		label = torch.tensor(label, dtype=torch.long)  # 确保label为long类型
		return img, label

# 准备数据
def load_data(csv_path, root_dir):
	# 改进的CSV读取方式
	df = pd.read_csv(csv_path, header=0)  # 使用第一行作为列头
	df = df.rename(columns={'ID': 'path', 'Label': 'label'})  # 规范列名
	
	# 数据类型转换
	convert_dict = {'path': str, 'label': int}
	df = df.astype(convert_dict)
	
	# 路径修正（根据实际文件结构可能需要调整）
	df['path'] = df['path'].apply(lambda x: os.path.join(root_dir, x))
	
	# 验证数据有效性
	print("正在验证数据完整性...")
	valid_samples = []
	for idx, row in tqdm(df.iterrows(), total=len(df)):
		if os.path.exists(row['path']):
			valid_samples.append(idx)
		else:
			print(f"警告：缺失文件 {row['path']}")
	df = df.loc[valid_samples]
	
	# 过滤无效类别（每个类别至少需要2个样本）
	label_counts = df['label'].value_counts()
	valid_labels = label_counts[label_counts >= 2].index
	df = df[df['label'].isin(valid_labels)]
	
	print(f"有效数据量: {len(df)}")
	print(f"有效类别数: {df['label'].nunique()}")
	
	# 转换为0-based标签
	
	# 分层划分数据集
	train_df, val_df = train_test_split(
		df,
		test_size=0.2,
		stratify=df['label'],
		random_state=CFG['seed']
	)
	return train_df, val_df

# 训练函数
def train_model(model, train_loader, test_loader):
	model = model.to(CFG['device'])
	criterion = nn.CrossEntropyLoss()
	optimizer = optim.Adam(model.parameters(), lr=CFG['lr'])
	
	best_acc = 0.0
	for epoch in range(CFG['num_epochs']):
		# 训练阶段
		model.train()
		running_loss = 0.0
		correct = 0
		total = 0
		
		pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{CFG["num_epochs"]} [Train]')
		for inputs, labels in pbar:
			inputs = inputs.to(CFG['device'])
			labels = labels.to(CFG['device'])
			
			optimizer.zero_grad()
			outputs = model(inputs)
			loss = criterion(outputs, labels)
			loss.backward()
			optimizer.step()
			
			running_loss += loss.item() * inputs.size(0)
			_, predicted = torch.max(outputs.data, 1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()
			
			pbar.set_postfix({
				'loss': loss.item(),
				'acc': f'{correct/total:.3f}'
			})
		
		# 验证阶段
		model.eval()
		test_correct = 0
		test_total = 0
		test_loss = 0.0
		
		with torch.no_grad():
			pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{CFG["num_epochs"]} [Test]')
			for inputs, labels in pbar:
				inputs = inputs.to(CFG['device'])
				labels = labels.to(CFG['device'])
				
				outputs = model(inputs)
				loss = criterion(outputs, labels)
				
				test_loss += loss.item() * inputs.size(0)
				_, predicted = torch.max(outputs.data, 1)
				test_total += labels.size(0)
				test_correct += (predicted == labels).sum().item()
				
				pbar.set_postfix({
					'acc': f'{test_correct/test_total:.3f}'
				})
		
		# 保存最佳模型
		epoch_acc = test_correct / test_total
		if epoch_acc > best_acc:
			best_acc = epoch_acc
			torch.save(model.state_dict(), 'best_model.pth')
			print(f'New best model saved with acc: {best_acc:.4f}')

# 预测并生成结果
def predict(model, test_loader):
	model.load_state_dict(torch.load('best_model.pth'))
	model.eval()
	
	filenames = []
	predictions = []
	
	with torch.no_grad():
		for inputs, _ in tqdm(test_loader, desc='Predicting'):
			inputs = inputs.to(CFG['device'])
			outputs = model(inputs)
			_, preds = torch.max(outputs, 1)
			predictions.extend((preds + 1).cpu().numpy())  # 转回1-based
	
	# 获取文件名
	test_df = test_loader.dataset.df
	filenames = test_df.iloc[:, 0].apply(lambda x: x.split('/')[-1]).tolist()
	
	# 写入文件
	with open('submission.txt', 'w') as f:
		for fn, pred in zip(filenames, predictions):
			f.write(f"{fn}\t{pred}\n")

if __name__ == '__main__':
	# 准备数据
	train_df, test_df = load_data(CFG['csv_name'], CFG['data_path'])
	
	# 创建数据集和数据加载器
	train_dataset = MedicineDataset(train_df, train_transform)
	test_dataset = MedicineDataset(test_df, test_transform)
	
	train_loader = DataLoader(
		train_dataset,
		batch_size=CFG['batch_size'],
		shuffle=True,
		num_workers=CFG['num_workers']
	)
	
	test_loader = DataLoader(
		test_dataset,
		batch_size=CFG['batch_size'],
		shuffle=False,
		num_workers=CFG['num_workers']
	)
	
	# 初始化模型
	model = models.resnet50(pretrained=True)
	model.fc = nn.Linear(model.fc.in_features, CFG['num_classes'])
	
	# 训练模型
	train_model(model, train_loader, test_loader)
	
	# 生成预测结果
	predict(model, test_loader)


正在验证数据完整性...


100%|██████████| 167017/167017 [00:10<00:00, 16271.12it/s]
c:\Users\wyz20\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wyz20\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


有效数据量: 167017
有效类别数: 879


Epoch 1/30 [Train]:   5%|▌         | 449/8351 [01:49<34:41,  3.80it/s, loss=6.76, acc=0.001]  c:\Users\wyz20\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1/30 [Test]: 100%|██████████| 2088/2088 [12:44<00:00,  2.73it/s, acc=0.044]


New best model saved with acc: 0.0436


...

New best model saved with acc: 0.5436


Epoch 28/30 [Test]: 100%|██████████| 2088/2088 [13:36<00:00,  2.56it/s, acc=0.549]


New best model saved with acc: 0.5487


Epoch 30/30 [Test]: 100%|██████████| 2088/2088 [10:52<00:00,  3.20it/s, acc=0.545]


AttributeError: type object 'tqdm' has no attribute 'tqdm'

In [11]:
def generate_predictions(model, test_dir, transform):
	model.eval()
	predictions = []
	
	# 获取测试图片列表
	test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) 
				  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
	
	# 创建测试数据加载器
	class TestDataset(Dataset):
		def __init__(self, image_paths, transform=None):
			self.image_paths = image_paths
			self.transform = transform
			
		def __len__(self):
			return len(self.image_paths)
		
		def __getitem__(self, idx):
			image = Image.open(self.image_paths[idx]).convert('RGB')
			if self.transform:
				image = self.transform(image)
			return image, os.path.basename(self.image_paths[idx])
	
	test_dataset = TestDataset(test_images, transform=transform)
	test_loader = DataLoader(test_dataset, batch_size=CFG['batch_size'], shuffle=False)
	
	# 进行预测
	with torch.no_grad():
		for images, filenames in tqdm(test_loader, desc='Predicting'):
			images = images.to(CFG['device'])
			outputs = model(images)
			_, preds = torch.max(outputs.logits, 1)
			
			for fn, pred in zip(filenames, preds.cpu().numpy()):
				predictions.append((fn, pred + 1))  # 转换回1-based
	
	# 保存结果
	with open('submission.txt', 'w') as f:
		for fn, pred in predictions:
			f.write(f'{fn}\t{pred}\n')


In [13]:

# 生成提交文件（假设测试图片在test目录下）
# 修正 outputs.logits 为 outputs
def generate_predictions(model, test_dir, transform):
	model.eval()
	predictions = []
	
	# 获取测试图片列表
	test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) 
				  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
	
	# 创建测试数据加载器
	class TestDataset(Dataset):
		def __init__(self, image_paths, transform=None):
			self.image_paths = image_paths
			self.transform = transform
			
		def __len__(self):
			return len(self.image_paths)
		
		def __getitem__(self, idx):
			image = Image.open(self.image_paths[idx]).convert('RGB')
			if self.transform:
				image = self.transform(image)
			return image, os.path.basename(self.image_paths[idx])
	
	test_dataset = TestDataset(test_images, transform=transform)
	test_loader = DataLoader(test_dataset, batch_size=CFG['batch_size'], shuffle=False)
	
	# 进行预测
	with torch.no_grad():
		for images, filenames in tqdm(test_loader, desc='Predicting'):
			images = images.to(CFG['device'])
			outputs = model(images)
			_, preds = torch.max(outputs, 1)
			
			for fn, pred in zip(filenames, preds.cpu().numpy()):
				predictions.append((fn, pred + 1))  # 转换回1-based
	
	# 保存结果
	with open('submission.txt', 'w') as f:
		for fn, pred in predictions:
			f.write(f'{fn}\t{pred}\n')

generate_predictions(model, 'chinese-medicine-image/test', test_transform)


Predicting:  12%|█▏        | 558/4694 [01:23<11:56,  5.77it/s]c:\Users\wyz20\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Predicting: 100%|██████████| 4694/4694 [11:41<00:00,  6.69it/s]
